<div style="padding: 20px; background-color: #f5f5f5; border-radius: 5px; border: 1px solid #ddd;">
    <h3 style="color: #333; margin-top: 0;">✓  Результаты анализа данных</h3>
    <p style="color: #666;">Интерактивный дашборд содержит полную визуализацию результатов анализа, включая:</p>
    <ul style="color: #666;">
        <li>Ключевые метрики эффективности</li>
        <li>Динамику продаж</li>
        <li>Анализ категорий</li>
        <li>Детальные графики и диаграммы</li>
    </ul>
    <p style="margin-top: 15px;">
        <a href="https://anfisaanalytics.github.io/new_river/" 
           style="background-color: #0066cc; color: white; padding: 8px 15px; text-decoration: none; border-radius: 4px; display: inline-block;"
           target="_blank">
           Просмотреть дашборд
        </a>
    </p>
</div>


## В данном проекте выполнен комплексный анализ данных с использованием следующих ключевых методов и инструментов:

#### Технический стек:
- Python (pandas, numpy)
- JavaScript (Chart.js)
- HTML/CSS

#### Основные компоненты анализа:
1. **Обработка данных**
   - Агрегация данных по заказам
   - Расчет ключевых метрик продаж
   - Анализ категорий товаров

2. **Визуализация**
   - Динамика заказов и выручки
   - Распределение по категориям (pie chart)
   - Анализ прибыльности товаров
   - Мониторинг стоимости хранения

3. **Метрики эффективности**
   - Общая выручка
   - Средняя маржинальность
   - Количество заказов
   - Средний чек


In [1]:
import pandas as pd
import json
from datetime import datetime
import numpy as np
import pytz
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_and_prepare_data():
    """Загрузка и подготовка данных"""
    # Загрузка датасетов
    orders_df = pd.read_csv('OrdersStatistic.csv')
    nomenclature_df = pd.read_csv('Nomenclature.csv')
    commission_df = pd.read_csv('ComissionBySubject.csv')
    warehouse_df = pd.read_csv('WarhouseName.csv')
    storage_df = pd.read_csv('StorageAmount.csv')
    
    # Конвертация дат
    orders_df['date'] = pd.to_datetime(orders_df['date'], format='ISO8601')
    storage_df['Date'] = pd.to_datetime(storage_df['Date'])
    
    return orders_df, nomenclature_df, commission_df, warehouse_df, storage_df

In [3]:
def calculate_key_metrics(orders_df, nomenclature_df, commission_df):
    """Расчет ключевых метрик с улучшенной обработкой данных"""
    # Фильтрация валидных заказов и проверка на отрицательные значения
    valid_orders = orders_df[
        (orders_df['isCancel'] == 0) & 
        (orders_df['orderType'] == 'Клиентский') &
        (orders_df['SoldPrice'] > 0)  # Исключаем отрицательные цены
    ]
    
    total_revenue = valid_orders['SoldPrice'].sum()
    total_orders = len(valid_orders)
    avg_order_value = total_revenue / total_orders if total_orders > 0 else 0
    
    # Улучшенное объединение данных с проверкой на наличие всех необходимых столбцов
    merged_data = valid_orders.merge(
        nomenclature_df[['NmId', 'SubjectName', 'CostPrice']], 
        on='NmId',
        how='left'
    ).merge(
        commission_df[['SubjectName', 'Commission']], 
        on='SubjectName',
        how='left'
    )
    
    # Заполняем пропущенные значения средними, чтобы избежать ошибок в расчетах
    merged_data['CostPrice'] = merged_data['CostPrice'].fillna(merged_data['CostPrice'].mean())
    merged_data['Commission'] = merged_data['Commission'].fillna(merged_data['Commission'].mean())
    
    # Улучшенный расчет маржи с учетом комиссии
    merged_data['margin'] = (
        (merged_data['SoldPrice'] * (1 - merged_data['Commission']/100) - 
         merged_data['CostPrice']) / merged_data['SoldPrice'] * 100
    )
    
    avg_margin = merged_data['margin'].mean()
    
    return {
        'total_revenue': total_revenue,
        'total_orders': total_orders,
        'avg_order_value': avg_order_value,
        'avg_margin': avg_margin,
        'data_quality': {
            'missing_cost_price': merged_data['CostPrice'].isnull().sum(),
            'missing_commission': merged_data['Commission'].isnull().sum(),
            'negative_margin_count': (merged_data['margin'] < 0).sum()
        }
    }

In [4]:
def create_orders_chart(orders_df):
    """Создание комбинированного графика заказов и выручки"""
    daily_orders = orders_df[orders_df['isCancel'] == 0].groupby(
        orders_df['date'].dt.date
    ).agg({
        'SoldPrice': 'sum',
        'NmId': 'count'
    }).reset_index()
    
    daily_orders.columns = ['date', 'revenue', 'orders']
    
    orders_data = daily_orders['orders'].astype(int).tolist()
    revenue_data = daily_orders['revenue'].astype(float).tolist()
    dates = [d.strftime('%Y-%m-%d') for d in daily_orders['date']]

    return {
        'type': 'bar',
        'data': {
            'labels': dates,
            'datasets': [
                {
                    'type': 'line',
                    'label': 'Количество заказов',
                    'data': orders_data,
                    'borderColor': '#817B7B',  # -------- линии заказов
                    'backgroundColor': 'rgba(76, 175, 80, 0.0)',
                    'yAxisID': 'y',
                    'tension': 0.4,
                    'order': 0
                },
                {
                    'type': 'bar',
                    'label': 'Выручка',
                    'data': revenue_data,
                    'backgroundColor': '#0E9AA7',  # ----------------- цвет для столбцов выручки
                    'borderColor': 'rgba(76, 175, 80, 0.0)',
                    'borderWidth': 1,
                    'yAxisID': 'y1',
                    'order': 1
                }
            ]
        },
        'options': {
            'responsive': True,
            'maintainAspectRatio': False,
            'interaction': {
                'mode': 'index',
                'intersect': False,
            },
            'plugins': {
                'title': {
                    'display': True,
                    'text': 'Динамика заказов и выручки'
                },
                'tooltip': {
                    'mode': 'index',
                    'intersect': False
                },
                'datalabels': {
                    'display': False}
            },
            'scales': {
                'y': {
                    'type': 'linear',
                    'display': True,
                    'position': 'left',
                    'title': {
                        'display': True,
                        'text': 'Количество заказов'
                    },
                    'grid': {
                        'drawOnChartArea': False
                    }
                },
                'y1': {
                    'type': 'linear',
                    'display': True,
                    'position': 'right',
                    'title': {
                        'display': True,
                        'text': 'Выручка (₽)'
                    },
                    'grid': {
                        'drawOnChartArea': True
                    }
                }
            }
        }
    }

In [5]:
def create_profit_chart(orders_df, nomenclature_df, show_decimals=False):
    """
    Создание горизонтального графика топ-5 товаров по прибыли
    Args:
        orders_df: DataFrame с заказами
        nomenclature_df: DataFrame с номенклатурой
        show_decimals: если True, показывать десятичные знаки, если False - округлять до целых
    """
    merged_data = orders_df[orders_df['isCancel'] == 0].merge(
        nomenclature_df[['NmId', 'SubjectName', 'CostPrice', 'BrandName']], 
        on='NmId'
    )
    
    product_profit = merged_data.groupby(['NmId', 'SubjectName', 'BrandName']).agg({
        'SoldPrice': 'sum',
        'CostPrice': lambda x: x.iloc[0] * len(x)
    }).reset_index()
    
    product_profit['profit'] = product_profit['SoldPrice'] - product_profit['CostPrice']
    top_products = product_profit.nlargest(5, 'profit')
    
    labels = [
        f"{row['SubjectName']}\n{row['BrandName']}\n(арт. {row['NmId']})" 
        for _, row in top_products.iterrows()
    ]
    profit_data = top_products['profit'].astype(float).tolist()
    
    return {
        'type': 'bar',
        'data': {
            'labels': labels,
            'datasets': [{
                'label': 'Прибыль',
                'data': profit_data,
                'backgroundColor': '#A4D4D8',
                'borderColor': '#21585D',
                'borderWidth': 1
            }]
        },
        'options': {
            'indexAxis': 'y',
            'responsive': True,
            'maintainAspectRatio': False,
            'plugins': {
                'datalabels': {
                    'display': False
                },
                'title': {
                    'display': True,
                    'text': 'Топ-5 товаров по прибыли'
                },
                'legend': {
                    'position': 'top',
                }
            },
            'scales': {
                'x': {
                    'beginAtZero': True,
                    'title': {
                        'display': True,
                        'text': 'Прибыль (₽)'
                    }
                }
            }
        }
    }

In [6]:
def create_storage_chart(storage_df):
    """Создание графика стоимости хранения с улучшенной обработкой дат"""
    # Проверяем и заполняем пропущенные даты
    date_range = pd.date_range(
        start=storage_df['Date'].min(),
        end=storage_df['Date'].max(),
        freq='D'
    )
    
    # Создаем полный датафрейм со всеми датами
    full_date_df = pd.DataFrame({'Date': date_range})
    
    # Объединяем с существующими данными
    daily_storage = full_date_df.merge(
        storage_df.groupby('Date')['WarehousePrice'].sum().reset_index(),
        on='Date',
        how='left'
    )
    
    # Заполняем пропуски предыдущими значениями или нулями
    daily_storage['WarehousePrice'] = daily_storage['WarehousePrice'].fillna(method='ffill').fillna(0)
    
    storage_data = daily_storage['WarehousePrice'].astype(float).tolist()
    dates = [d.strftime('%Y-%m-%d') for d in daily_storage['Date']]

    return {
        'type': 'line',
        'data': {
            'labels': dates,
            'datasets': [{
                'label': 'Стоимость хранения',
                'data': storage_data,
                'borderColor': '#FE8A71',
                'backgroundColor': 'rgba(156, 39, 176, 0.0)',
                'tension': 0.4
            }]
        },
        'options': {
            'responsive': True,
            'maintainAspectRatio': False,
            'plugins': {
                'title': {
                    'display': True,
                    'text': 'Динамика стоимости хранения'
                },
                'datalabels': {
                    'display': False
                }
            },
            'scales': {
                'y': {
                    'beginAtZero': True,
                    'title': {
                        'display': True,
                        'text': 'Стоимость (₽)'
                    }
                },
                'x': {
                    'title': {
                        'display': True,
                        'text': 'Дата'
                    }
                }
            }
        }
    }

In [7]:
def create_category_chart(orders_df, nomenclature_df, show_percentages=True):
    """
    Создание круговой диаграммы распределения по категориям
    Args:
        orders_df: DataFrame с заказами
        nomenclature_df: DataFrame с номенклатурой
        show_percentages: если True, показывать проценты, если False - количество
    """
    merged_data = orders_df[orders_df['isCancel'] == 0].merge(
        nomenclature_df[['NmId', 'SubjectName']], 
        on='NmId'
    )
    
    category_sales = merged_data.groupby('SubjectName').size()
    top_categories = category_sales.nlargest(4)
    other_sales = category_sales[4:].sum()
    
    total_sales = category_sales.sum()
    
    labels = list(top_categories.index)
    values = [int(x) for x in top_categories.values]
    percentages = [(x / total_sales) * 100 for x in values]
    
    labels.append('Другие категории')
    values.append(int(other_sales))
    percentages.append((other_sales / total_sales) * 100)

    # Выбираем данные для отображения в зависимости от параметра
    display_data = percentages if show_percentages else values
    
    return {
        'type': 'doughnut',
        'data': {
            'labels': labels,
            'datasets': [{
                'data': display_data,
                'backgroundColor': [
                    '#F6CD61', '#A4D4D8', '#0E9AA7', '#FE8A71', '#817B7B'
                ],
                'borderColor': [
                    '#F6CD61', '#A4D4D8', '#0E9AA7', '#FE8A71', '#817B7B'
                ],
                'borderWidth': 1
            }]
        },
        'options': {
            'responsive': True,
            'maintainAspectRatio': False,
            'plugins': {
                'datalabels': {
                    'display': False
                },
                'title': {
                    'display': True,
                    'text': 'Распределение по категориям'
                },
                'legend': {
                    'position': 'top',
                }
            }
        }
    }

In [8]:
def get_top_products_table(orders_df, nomenclature_df, commission_df):
    """Создание таблицы топ-10 товаров с улучшенными расчетами"""
    merged_data = orders_df[orders_df['isCancel'] == 0].merge(
        nomenclature_df[['NmId', 'SubjectName', 'CostPrice', 'BrandName']], 
        on='NmId'
    ).merge(
        commission_df[['SubjectName', 'Commission']], 
        on='SubjectName'
    )
    
    # Группировка с дополнительными метриками
    product_metrics = merged_data.groupby(['NmId', 'SubjectName', 'BrandName']).agg({
        'SoldPrice': ['sum', 'count', 'mean'],  # Добавляем среднюю цену продажи
        'CostPrice': 'first',
        'Commission': 'first',
        'date': [
            lambda x: (x.max() - x.min()).days + 1,
            'min',
            'max'
        ]
    }).reset_index()
    
    # Переименовываем столбцы для удобства
    product_metrics.columns = [
        'NmId', 'SubjectName', 'BrandName', 'Revenue', 'Orders', 
        'AvgPrice', 'CostPrice', 'Commission', 'Days', 'FirstSale', 'LastSale'
    ]
    
    # Улучшенный расчет маржи с учетом комиссии
    product_metrics['Margin'] = (
        (product_metrics['Revenue'] * (1 - product_metrics['Commission']/100) - 
         product_metrics['CostPrice'] * product_metrics['Orders']) / 
        product_metrics['Revenue'] * 100
    )
    
    # Улучшенный расчет заказов в день
    product_metrics['OrdersPerDay'] = product_metrics['Orders'] / product_metrics['Days'].clip(lower=1)
    
    # Добавляем дополнительные метрики
    product_metrics['DaysSinceLastSale'] = (
        pd.Timestamp.now() - product_metrics['LastSale']
    ).dt.days
    
    # Добавляем ROI
    product_metrics['ROI'] = (
        (product_metrics['Revenue'] * (1 - product_metrics['Commission']/100) - 
         product_metrics['CostPrice'] * product_metrics['Orders']) / 
        (product_metrics['CostPrice'] * product_metrics['Orders']) * 100
    )
    
    return product_metrics.nlargest(10, 'Revenue')

In [9]:
def format_number(value):
    return f"{value:,.0f}".replace(',', ' ')

In [10]:
def generate_html_report():
    """Генерация HTML отчета"""
    # Загрузка данных
    orders_df, nomenclature_df, commission_df, warehouse_df, storage_df = load_and_prepare_data()
    
    # Расчет метрик
    metrics = calculate_key_metrics(orders_df, nomenclature_df, commission_df)
    
    # Создание графиков
    orders_chart = create_orders_chart(orders_df)
    profit_chart = create_profit_chart(orders_df, nomenclature_df, show_decimals=False)
    storage_chart = create_storage_chart(storage_df)
    category_chart = create_category_chart(orders_df, nomenclature_df, show_percentages=True)
    
    # Получение таблицы топ товаров
    top_products = get_top_products_table(orders_df, nomenclature_df, commission_df)
    
    # Генерация HTML
    html_template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Анализ данных маркетплейса</title>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <!-- Сначала подключаем Chart.js -->
        <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.1/dist/chart.umd.min.js"></script>
        <!-- Затем подключаем плагин datalabels -->
        <script src="https://cdn.jsdelivr.net/npm/chartjs-plugin-datalabels@2.2.0"></script>
        <!-- И регистрируем плагин -->
        <script>
            Chart.register(ChartDataLabels);
        </script>
        <style>
            :root {{
                /* Default theme colors */
                --chart-color-1: #2563eb;
                --chart-color-1-rgb: 37, 99, 235;
                --chart-color-1-dark: #1d4ed8;
                --chart-color-2: #3b82f6;
                --chart-color-2-rgb: 59, 130, 246;
                --chart-color-3: #60a5fa;
                --chart-color-4: #93c5fd;
                --chart-color-5: #bfdbfe;
            }}

            /* Theme variations */
            [data-theme="green"] {{
                --chart-color-1: #059669;
                --chart-color-1-rgb: 5, 150, 105;
                --chart-color-1-dark: #047857;
                --chart-color-2: #10b981;
                --chart-color-2-rgb: 16, 185, 129;
                --chart-color-3: #34d399;
                --chart-color-4: #6ee7b7;
                --chart-color-5: #a7f3d0;
            }}

            [data-theme="purple"] {{
                --chart-color-1: #7c3aed;
                --chart-color-1-rgb: 124, 58, 237;
                --chart-color-1-dark: #6d28d9;
                --chart-color-2: #8b5cf6;
                --chart-color-2-rgb: 139, 92, 246;
                --chart-color-3: #a78bfa;
                --chart-color-4: #c4b5fd;
                --chart-color-5: #ddd6fe;
            }}

            body {{
                font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif;
                margin: 0;
                padding: 20px 22rem;
                background: #f5f5f5;
            }}

            .dashboard {{
                margin: 0 auto;
            }}

            .header {{
                background: #fff;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                margin-bottom: 20px;
                display: flex;
                justify-content: space-between;
                align-items: center;
                flex-wrap: wrap;
                gap: 20px;
            }}

            .theme-selector {{
                display: flex;
                gap: 10px;
            }}

            .theme-btn {{
                padding: 8px 16px;
                border: none;
                border-radius: 4px;
                cursor: pointer;
                font-weight: 500;
            }}

            .metrics-container {{
                display: grid;
                grid-template-columns: 30% auto;
                gap: 20px;
                margin-bottom: 20px;
            }}

            @media (max-width: 768px) {{
                .metrics-container {{
                    grid-template-columns: 1fr;
                }}
            }}

            .metrics-column {{
                background: #fff;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }}
            .metrics-column>ul>li{{
                line-height:1.9}}
            .metric-item {{
                margin-bottom: 20px;
            }}

            .metric-value {{
                font-size: 24px;
                font-weight: bold;
                color: #165e84f2;
            }}

            .metric-label {{
                color: #666;
                font-size: 14px;
            }}

            .chart-grid {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
                gap: 20px;
            }}

            .chart-container {{
                background: #fff;
                padding: 20px;
                margin: 1rem 0;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                height: 400px;
            }}

            
           
            .table-container {{
                background: #fff;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                margin-top: 20px;
                overflow-x: auto;
            }}

            table {{
                width: 100%;
                border-collapse: collapse;
                min-width: 600px;
            }}

            th, td {{
                padding: 12px;
                text-align: left;
                border-bottom: 1px solid #eee;
            }}

            th {{
                background: #f8fafc;
                font-weight: 600;
            }}

            @media (max-width: 1500px) {{
             body{{padding: 20px;
             }}}}
            @media (max-width: 768px) {{
                body {{
                    padding: 10px;
                }}
                
                .header {{
                    padding: 15px;
                }}

                .chart-container {{
                    height: 300px;
                }}
            }}
        </style>
    </head>
    <body>
        <div class="dashboard">
            <div class="header">
                <div>
                    <h1>Анализ данных маркетплейса</h1>
                    <p>Отчёт сгенерирован - {datetime.now(pytz.UTC).astimezone(pytz.timezone('Europe/Moscow')).strftime("%d.%m.%Y %H:%M %Z")}</p>
                </div>
                
            </div>

            <div class="metrics-container">
                <div class="metrics-column">
                    <h2>Ключевые значения</h2>
                    <div class="metric-item">
                        <div class="metric-value">₽ {format_number(metrics['total_revenue'])}</div>
                        <div class="metric-label">Общая выручка</div>
                    </div>
                    <div class="metric-item">
                        <div class="metric-value">{metrics['avg_margin']:.1f}%</div>
                        <div class="metric-label">Средняя маржинальность</div>
                    </div>
                    <div class="metric-item">
                        <div class="metric-value">{format_number(metrics['total_orders'])}</div>
                        <div class="metric-label">Количество заказов</div>
                    </div>
                    <div class="metric-item">
                        <div class="metric-value">₽ {metrics['avg_order_value']:,.0f}</div>
                        <div class="metric-label">Средний чек</div>
                    </div>
                </div>

                <div class="metrics-column">
                    <h2>Ключевые выводы</h2>
                    <ul>
                        <li><strong>Динамика заказов:</strong> {
                            'Стабильный рост' if metrics['total_orders'] > 0 else 'Недостаточно данных'
                        }</li>
                        <li><strong>Маржа:</strong> {metrics['avg_margin']:.1f}%</li>
                        <li><strong>Средний чек:</strong> ₽ {metrics['avg_order_value']: .0f}</li>
                        <li><strong>Эффективность продаж:</strong> <br /> Общее кол.заказов {format_number(metrics['total_orders'])} > выручка {format_number(metrics['total_revenue'])} ₽</li>
                    </ul>
                </div>
            </div>

                <div class="chart-container">
                    <canvas id="ordersChart"></canvas>
                </div>
                <div class="chart-container">
                    <canvas id="profitChart"></canvas>
                </div>
                <div class="chart-container">
                    <canvas id="storageChart"></canvas>
                </div>
                <div class="chart-container">
                    <canvas id="categoryChart"></canvas>
                </div>
            
            <div class="table-container">
                <h2>Топ-10 товаров по выручке</h2>
                <table>
                    <thead>
                        <tr>
                            <th>Артикул</th>
                            <th>Категория</th>
                            <th>Выручка</th>
                            <th>Маржа</th>
                            <th>Заказов/день</th>
                        </tr>
                    </thead>
                    <tbody>
                        {''.join(f"""
                        <tr>
                            <td>{row['NmId']}</td>
                            <td>{row['SubjectName']}</td>
                            <td>₽ {row['Revenue']:,.0f}</td>
                            <td>{row['Margin']:.1f}%</td>
                            <td>{row['OrdersPerDay']:.1f}</td>
                        </tr>
                        """ for _, row in top_products.iterrows())}
                    </tbody>
                </table>
            </div>
        </div>

        <script>
            // Функция изменения темы
            function setTheme(theme) {{
                document.body.setAttribute('data-theme', theme);
                // Перерисовка графиков
                Object.values(charts).forEach(chart => chart.update());
            }}

            // Создание графиков
            const ordersChartData = {json.dumps(orders_chart)};
            const profitChartData = {json.dumps(profit_chart)};
            const storageChartData = {json.dumps(storage_chart)};
            const categoryChartData = {json.dumps(category_chart)};

            const charts = {{}};

            // Функция для создания графика
            function createChart(canvasId, chartConfig) {{
                const ctx = document.getElementById(canvasId).getContext('2d');
                charts[canvasId] = new Chart(ctx, chartConfig);
                return charts[canvasId];
            }}

            // Создание всех графиков
            createChart('ordersChart', ordersChartData);
            createChart('profitChart', profitChartData);
            createChart('storageChart', storageChartData);
            createChart('categoryChart', categoryChartData);
        </script>
    </body>
    </html>
    """
    
    # Сохранение HTML отчета
    with open(f'report_{datetime.now(pytz.UTC).astimezone(pytz.timezone('Europe/Moscow')).strftime("%d.%m.%Y_%H:%M")}.html', 'w', encoding='utf-8') as f:
        f.write(html_template)

In [11]:
if __name__ == "__main__":
    generate_html_report()
    print('Отчет успешно сгенерирован') #+

Отчет успешно сгенерирован
